In [1]:
import glob
import logging
import fairing
GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
DOCKER_REGISTRY = 'gcr.io/{}'.format(GCP_PROJECT) # every Google Cloud Platform project comes with a private Docker registry
base_image = "{}/{}".format(DOCKER_REGISTRY, "fairing:latest")
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR) # suppress nagging bug about a library incompatibility

## Instructions

**Authenticate** with the docker repo first:

```bash
gcloud auth configure-docker```

If using TPUs please also authorize Cloud TPU to access your project as [described here](https://cloud.google.com/ml-engine/docs/tensorflow/using-tpus#authorize_your_to_access_your_project).

## Build a base image to work with fairing


In [2]:
!cat Dockerfile

FROM gcr.io/deeplearning-platform-release/tf-gpu.1-13
ENV LC_ALL C.UTF-8
ENV LANG C.UTF-8
RUN pip install tensorflow-gpu==2.0.0-beta1

In [ ]:
!docker build . -t {base_image}

In [ ]:
!docker push {base_image}

## Start an AI Platform job

In [15]:
additional_files = '' # If your code requires additional files, you can specify them here (or include everything in the current folder with glob.glob('./**', recursive=True))
# If your code does not require any dependencies or config changes, you can directly start from an official Tensorflow docker image
#fairing.config.set_builder('docker', registry=DOCKER_REGISTRY, base_image='gcr.io/deeplearning-platform-release/tf-gpu.1-13')

# base image
fairing.config.set_builder('docker', registry=DOCKER_REGISTRY, base_image=base_image)
# AI Platform job hardware config
fairing.config.set_deployer('gcp', job_config={'trainingInput': {'scaleTier': 'BASIC_GPU'}})
# input and output notebooks
fairing.config.set_preprocessor('full_notebook',
                                notebook_file="05_MNIST_TF20Keras_Tensorboard_playground.ipynb",
                                input_files=additional_files,
                                output_file='gs://ml1-demo-martin/fairing-output/mnistTF20beta1-001.ipynb')


# GPU settings for single K80, single p100 respectively
# job_config={'trainingInput': {'scaleTier': 'BASIC_GPU'}}
# job_config={'trainingInput': {'scaleTier': 'CUSTOM', 'masterType': 'standard_p100'}}

# These job_config settings work for TPUv2, TPUv3 and TPUv2 respectively.
#job_config={'trainingInput': {'scaleTier': 'BASIC_GPU'}}
#job_config={'trainingInput': {'scaleTier': 'CUSTOM', 'masterType': 'standard', 'workerType': 'cloud_tpu_v3', 'workerCount': 1}}
#job_config={'trainingInput': {'scaleTier': 'CUSTOM', 'masterType': 'n1-standard-8', 'workerType': 'cloud_tpu', 'workerCount': 1,
#                                               'workerConfig': {'accelerator_config': {'type': 'TPU_V2','count': 8}}}})

In [ ]:
fairing.config.run()

## License



---


author: Martin Gorner<br>
twitter: @martin_gorner


---


Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose
